# Unit 2 - Cost of Test

Group members:

- Ole Magnus Laerum
- Piotr Franc

# Task Description
Consider the dataset Breastcancer with the following partition train/test. Consider the test costs for each attribute given by the array testcosts (the first position of the array corresponds to the first attribute, CL.thickness, the second to Cell.size..). Consider also misclassification costs where the cost of predicting “benign” when the actual class is “malignant” is 20. The cost of predicting “malignant” when the actual class is “benign” is 4. Hits cost 0.

Try to minimise the global cost of your model. Draw a plot of test cost versus missclassification cost of the different options explored.

In [1]:
print("Hello World")

Hello World
